In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pandas as pd
sys.path.append('auto_text_classifier')
from atc.models.aml import AML
from atc.utils.data_utils import load_df,init_dir
from atc.utils.data_utils import get_data_report
import numpy as np
import json
import random
from utils.data_proprocess import clean_raw_sentence

Using TensorFlow backend.
/root/anaconda3/envs/4s_dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/anaconda3/envs/4s_dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/anaconda3/envs/4s_dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/anaconda3/envs/4s_dev/lib/python3.7

## 原始数据合并

In [3]:
keep_cols = ['Id', 'Dialogue_id', 'Utterance_id', 'Sentence', 'Speaker', 'Label']

In [4]:
def clean_df(df):
    df = df.dropna().copy()
    for int_col in ['Dialogue_id','Id','Label','Utterance_id']:
        print(int_col)
        df[int_col] = df[int_col].apply(int)
    df['Speaker'] = df['Speaker'].apply(lambda x: str(x).strip())
    df['Sentence'] = df['Sentence'].apply(lambda x: str(x).strip())
    df['Sentence'] = df['Sentence'].apply(clean_raw_sentence)
    df = df[df['Sentence']!=""]
    df = df[df['Label'].isin([0,1])]
    return df

### charlie

In [5]:
data_dir = 'data/官方数据/train/'
df_list = []
for fname in os.listdir(data_dir):
    if not '.csv' in fname:
        continue
    df = pd.read_csv(os.path.join(data_dir, fname))
    new_cols = [x.capitalize() for x in list(df.columns)]
    df.columns = new_cols
    if df.shape[1] != 6 or len(set(new_cols) & set(keep_cols)) != 6:
        print(fname)
    df['fname'] = fname
    df_list.append(df)
df = pd.concat(df_list)  # [keep_cols]
df = clean_df(df)
df.shape

Dialogue_id
Id
Label
Utterance_id


(18855, 7)

In [6]:
df[df['Sentence'].isna()]

,Id,Dialogue_id,Utterance_id,Sentence,Speaker,Label,fname


In [7]:
df.shape

(18855, 7)

In [8]:
df['Label'].value_counts(normalize=True)

0    0.846142
1    0.153858
Name: Label, dtype: float64

In [9]:
df['Dialogue_id_old'] = df['Dialogue_id']
df['Dialogue_id'] = df.apply(lambda x:f"{x['fname']}_{x['Dialogue_id']}",axis=1)

In [10]:
df.to_csv('data/after_preprocess/task1_charlie_train.csv',index=False)

### task1dev数据

In [11]:
data_dir = 'data/官方数据/dev/task1 dev/'
df_list = []
for fname in os.listdir(data_dir):
    if not '.csv' in fname:
        continue
    df = pd.read_csv(os.path.join(data_dir, fname))
    df['Sentence'] = df['Sentence'].apply(clean_raw_sentence)
    new_cols = [x.capitalize() for x in list(df.columns)]
    df.columns = new_cols
    df['fname'] = fname
    df_list.append(df)
df = pd.concat(df_list)  # [keep_cols]
# df = clean_df(df)
df.shape

(2811, 7)

In [12]:
df['Dialogue_id_old'] = df['Dialogue_id']
df['Dialogue_id'] = df.apply(lambda x:f"{x['fname']}_{x['Dialogue_id']}",axis=1)

In [13]:
df

,Id,Dialogue_id,Utterance_id,Sentence,Speaker,Label,fname,Dialogue_id_old
0,0,Charlie_S01E08.csv_1,0,"Well, Charlie, today is a very special day.",Teddy,0.0,Charlie_S01E08.csv,1
1,1,Charlie_S01E08.csv_1,1,It's your first birthday! The whole family's e...,Teddy,0.0,Charlie_S01E08.csv,1
2,2,Charlie_S01E08.csv_1,2,"Happy Birthday, sweetie.",Amy,0.0,Charlie_S01E08.csv,1
3,3,Charlie_S01E08.csv_1,3,"I love you, baby girl.",Bob,0.0,Charlie_S01E08.csv,1
4,4,Charlie_S01E08.csv_1,4,You're the best.,PJ,0.0,Charlie_S01E08.csv,1
...,...,...,...,...,...,...,...,...
301,301,Charlie_S02E05.csv_17,0,"So elegant, so poised.",Others,0.0,Charlie_S02E05.csv,17
302,302,Charlie_S02E05.csv_17,1,Such sophistication.,Others,0.0,Charlie_S02E05.csv,17
303,303,Charlie_S02E05.csv_17,2,"Our finest pupil ever, Mrs.Krump.",Others,0.0,Charlie_S02E05.csv,17
304,304,Charlie_S02E05.csv_17,3,Uh-oh.,Charlie,0.0,Charlie_S02E05.csv,17


In [14]:
df['Label'].value_counts()

0.0    2448
1.0     363
Name: Label, dtype: int64

In [15]:
df.to_csv('data/after_preprocess/task1_charlie_dev.csv',index=False)

### task1 test

In [16]:
data_dir = 'data/官方数据/test/task1_test/'
df_list = []
for fname in os.listdir(data_dir):
    if not '.csv' in fname:
        continue
    df = pd.read_csv(os.path.join(data_dir, fname))
    df['Sentence'] = df['Sentence'].apply(clean_raw_sentence)
    new_cols = [x.capitalize() for x in list(df.columns)]
    df.columns = new_cols
    df['fname'] = fname
    df_list.append(df)
df = pd.concat(df_list)  # [keep_cols]
# df = clean_df(df)
df.shape

(4122, 6)

In [17]:
df['Dialogue_id_old'] = df['Dialogue_id']
df['Dialogue_id'] = df.apply(lambda x:f"{x['fname']}_{x['Dialogue_id']}",axis=1)

In [18]:
df

,Id,Dialogue_id,Utterance_id,Sentence,Speaker,fname,Dialogue_id_old
0,7575,S04E16.csv_1,0,"[Music] Howdy, Duncans.",Beau,S04E16.csv,1
1,7576,S04E16.csv_1,1,Hey.,Bob,S04E16.csv,1
2,7577,S04E16.csv_1,2,"Hey, Beau.",Teddy,S04E16.csv,1
3,7578,S04E16.csv_1,3,"Morning, Sweetie.",Beau,S04E16.csv,1
4,7579,S04E16.csv_1,4,Hi.,Teddy,S04E16.csv,1
...,...,...,...,...,...,...,...
335,6251,S04E11.csv_17,335,And I have a question for you.,Bob,S04E11.csv,17
336,6252,S04E11.csv_17,336,How many times has this happened to you? This ...,Bob,S04E11.csv,17
337,6253,S04E11.csv_17,337,Bob's Butt Stuff.,Bob,S04E11.csv,17
338,6254,S04E11.csv_17,338,Is it a cream? Is it a salve? Can you grout yo...,Bob,S04E11.csv,17


In [19]:
df.to_csv('data/after_preprocess/task1_charlie_test.csv',index=False)

### task2dev数据

In [20]:
# S03E06，S03E07，S03E10和S03E11

In [21]:
data_dir = 'data/官方数据/test/task2_test/'
os.listdir(data_dir)

['S03E17.csv',
 'S03E18.csv',
 'S03E17.xlsx',
 'S03E15.csv',
 'S03E18.xlsx',
 'S03E15.xlsx',
 '.ipynb_checkpoints']

In [22]:
df_list = []
# for fname in ['S03E06.csv', 'S03E07.csv', 'S03E10.csv', 'S03E11.csv']:
for fname in ['S03E15.xlsx', 'S03E17.xlsx', 'S03E18.xlsx']:
    df = pd.read_excel(os.path.join(data_dir, fname))
    df['Sentence'] = df['Sentence'].apply(clean_raw_sentence)
    new_cols = [x.capitalize() for x in list(df.columns)]
    df.columns = new_cols
    df_list.append(df)
    df['fname'] = fname
df = pd.concat(df_list)  # [keep_cols]
# df = clean_df(df)
df.shape

(932, 8)

In [23]:
df['Label']

0      0
1      0
2      0
3      0
4      0
      ..
291    1
292    0
293    1
294    0
295    1
Name: Label, Length: 932, dtype: int64

In [24]:
df['Dialogue_id_old'] = df['Dialogue_id']
df['Dialogue_id'] = df.apply(lambda x:f"{x['fname']}_{x['Dialogue_id']}",axis=1)

In [25]:
df.to_csv('data/after_preprocess/task2_charlie_test.csv',index=False)

In [26]:
df['fname'].unique()

array(['S03E15.xlsx', 'S03E17.xlsx', 'S03E18.xlsx'], dtype=object)